## Step1

In [ ]:
import SimpleITK as sitk
import numpy as np
import os
from glob import glob
from tqdm import tqdm


def apply_windowing(image_array: np.ndarray, window_level: int, window_width: int) -> np.ndarray:
    """
    對 CT 影像陣列應用窗值變換，並將結果標準化到 [0, 1] 範圍。
    
    Args:
        image_array (np.ndarray): 原始 CT 影像的 NumPy 陣列 (HU 值)。
        window_level (int): 窗位 (Window Level, WL)。
        window_width (int): 窗寬 (Window Width, WW)。
    
    Returns:
        np.ndarray: 經過窗值處理並標準化後的影像陣列 [0, 1]。
    """
    # 根據 WL/WW 計算 HU 值的上下限
    lower_bound = window_level - (window_width / 2)
    upper_bound = window_level + (window_width / 2)
    
    # 1. 將超出範圍的值裁剪到邊界
    windowed_array = np.clip(image_array, lower_bound, upper_bound)
    
    # 2. 將窗值範圍內的 HU 值線性標準化到 [0, 1]
    normalized_array = (windowed_array - lower_bound) / window_width
    
    return normalized_array.astype(np.float32)


def process_dataset(input_dir: str, output_dir: str):
    """
    遍歷指定資料夾中的所有 CT 影像，將它們轉換為多通道格式並儲存。
    
    Args:
        input_dir (str): 包含原始 CT 影像的資料夾路徑
        output_dir (str): 儲存處理後的多通道影像的目標資料夾路徑
    """
    # 確保輸出資料夾存在，如果不存在則創建
    os.makedirs(output_dir, exist_ok=True)
    
    # 根據 nnU-Net 命名慣例，尋找所有 CT 影像檔案
    image_files = sorted(glob(os.path.join(input_dir, "patient*_0000.nii.gz")))
    
    if not image_files:
        print(f"錯誤：在 '{input_dir}' 中找不到任何影像檔案。")
        print("請確認您的路徑和檔案名稱是否正確 (例如 patient0001_0000.nii.gz)。")
        return
    
    print(f"找到 {len(image_files)} 個影像檔案，準備開始轉換...")
    
    # 定義三種窗值設定 (窗位 WL, 窗寬 WW)
    window_presets = {
        'soft_tissue': (40, 400),   # 通道 1: 突顯心肌與血液
        'vascular':    (150, 300),  # 通道 2: 突顯主動脈與血管結構
        'bone':        (300, 1500)  # 通道 3: 突顯高密度的鈣化點
    
    }
    
    # 使用 tqdm 顯示進度條
    for input_path in tqdm(image_files, desc="正在轉換影像"):
        try:
            # 讀取原始 NIfTI 影像
            original_image_sitk = sitk.ReadImage(input_path, sitk.sitkFloat32)
            original_array = sitk.GetArrayFromImage(original_image_sitk)
            
            # 初始化一個空列表，用來存放處理後的通道影像
            processed_images = []
            
            # 依序應用每種窗值設定
            for preset_name, (wl, ww) in window_presets.items():
                # 應用窗值處理
                channel_array = apply_windowing(original_array, wl, ww)
                
                # 將處理後的陣列轉換為 SimpleITK 影像物件
                channel_image = sitk.GetImageFromArray(channel_array)
                
                # 複製原始影像的元數據（spacing, origin, direction）
                channel_image.CopyInformation(original_image_sitk)
                
                # 加入到處理後的影像列表
                processed_images.append(channel_image)
            
            # 使用 sitk.Compose 將三個單通道影像合併為一個多通道影像
            # 這是 SimpleITK 推薦的方法，能正確處理元數據
            output_image_sitk = sitk.Compose(processed_images)
            
            # 構造輸出檔案路徑並儲存
            filename = os.path.basename(input_path)
            output_path = os.path.join(output_dir, filename)
            sitk.WriteImage(output_image_sitk, output_path)
            
        except Exception as e:
            print(f"\n處理檔案 {input_path} 時發生錯誤: {e}")
            continue
    
    print(f"\n所有影像已成功轉換！")
    print(f"新的多通道影像已儲存至: {output_dir}")
    
    # 驗證輸出：讀取第一個處理後的影像並顯示資訊
    if image_files:
        first_output = os.path.join(output_dir, os.path.basename(image_files[0]))
        if os.path.exists(first_output):
            verify_image = sitk.ReadImage(first_output)
            print(f"\n=== 驗證資訊 ===")
            print(f"影像大小: {verify_image.GetSize()}")
            print(f"通道數: {verify_image.GetNumberOfComponentsPerPixel()}")
            print(f"像素間距: {verify_image.GetSpacing()}")
            print(f"原點座標: {verify_image.GetOrigin()}")
            print(f"資料型別: {verify_image.GetPixelIDTypeAsString()}")


# --- 主程式執行區 ---
if __name__ == '__main__':
    # ---資料夾路徑 ---
    # 原始測試的 CT 影像的資料夾
    ct_images_dir_test = "/home/ncku/Desktop/multichannel_ct/imagesTs"
    # 儲存處理後的多通道影像的目標資料夾
    output_dir_test = "/home/ncku/Desktop/multichannel_ct/imagesTs_multichannel_3"
    # 執行處理流程
    process_dataset(ct_images_dir_test, output_dir_test)

    # ---資料夾路徑 ---
    # 原始訓練的 CT 影像的資料夾
    ct_images_dir_train = "/home/ncku/Desktop/multichannel_ct/imagesTr"
    # 儲存處理後的多通道影像的目標資料夾 
    output_dir_train = "/home/ncku/Desktop/multichannel_ct/imagesTr_multichannel_3"
    # 執行處理流程
    process_dataset(ct_images_dir_train, output_dir_train)

## Step2

In [ ]:
import SimpleITK as sitk
import numpy as np
import os
from glob import glob
from tqdm import tqdm


def split_multichannel_to_nnunet_format(input_dir: str, output_dir: str):
    """
    將多通道 NIfTI 影像拆分為 nnU-Net 格式的多個單通道檔案。
    
    例如：
    輸入：patient0001_0000.nii.gz (3 通道)
    輸出：
        patient0001_0000.nii.gz (通道 0)
        patient0001_0001.nii.gz (通道 1)
        patient0001_0002.nii.gz (通道 2)
    
    Args:
        input_dir (str): 包含多通道影像的資料夾路徑
        output_dir (str): 輸出資料夾路徑（nnU-Net 格式）
    """
    # 確保輸出資料夾存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 尋找所有多通道影像檔案
    image_files = sorted(glob(os.path.join(input_dir, "patient*_0000.nii.gz")))
    
    if not image_files:
        print(f"錯誤：在 '{input_dir}' 中找不到任何影像檔案。")
        return
    
    print("="*80)
    print("將多通道影像拆分為 nnU-Net 格式")
    print("="*80)
    print(f"輸入目錄: {input_dir}")
    print(f"輸出目錄: {output_dir}")
    print(f"找到 {len(image_files)} 個多通道影像檔案")
    print("="*80)
    
    success_count = 0
    fail_count = 0
    
    for input_path in tqdm(image_files, desc="拆分多通道影像"):
        try:
            # 提取患者 ID（例如 patient0001）
            filename = os.path.basename(input_path)
            patient_id = filename.replace("_0000.nii.gz", "")
            
            # 讀取多通道影像
            multi_img = sitk.ReadImage(input_path)
            num_channels = multi_img.GetNumberOfComponentsPerPixel()
            
            if num_channels == 1:
                print(f"\n警告：{filename} 是單通道影像，跳過...")
                continue
            
            # 拆分每個通道並儲存
            for ch_idx in range(num_channels):
                # 使用 VectorIndexSelectionCast 提取單一通道
                channel_img = sitk.VectorIndexSelectionCast(multi_img, ch_idx, sitk.sitkFloat32)
                
                # 構造輸出檔名：patient0001_0000.nii.gz, patient0001_0001.nii.gz, ...
                output_filename = f"{patient_id}_{ch_idx:04d}.nii.gz"
                output_path = os.path.join(output_dir, output_filename)
                
                # 儲存單通道影像
                sitk.WriteImage(channel_img, output_path)
            
            success_count += 1
            
        except Exception as e:
            print(f"\n處理檔案 {input_path} 時發生錯誤: {e}")
            fail_count += 1
            continue
    
    print("\n" + "="*80)
    print("拆分完成！")
    print("="*80)
    print(f"成功: {success_count} 個檔案")
    print(f"失敗: {fail_count} 個檔案")
    print(f"總計生成: {success_count * 3} 個單通道影像檔案")


# --- 主程式執行區 ---
if __name__ == '__main__':
    # --- 設定路徑 ---
    # 輸入：測試集多通道影像的資料夾
    multichannel_dir_test = "/home/ncku/Desktop/multichannel_ct/imagesTs_multichannel_3"
    # 輸出：nnU-Net 格式的影像資料夾
    # 這個路徑應該是你的 nnU-Net 資料集的 imagesTs 路徑
    nnunet_images_dir_test = "/home/ncku/Desktop/multi_nnunetv2/nnUNet/nnUNet_raw/Dataset520_trueLumen/imagesTs"
    # 執行拆分
    split_multichannel_to_nnunet_format(multichannel_dir_test, nnunet_images_dir_test)

    # --- 設定路徑 ---
    # 輸入：訓練集多通道影像的資料夾
    multichannel_dir_train = "/home/ncku/Desktop/multichannel_ct/imagesTr_multichannel_3"
    # 輸出：nnU-Net 格式的影像資料夾
    # 這個路徑應該是你的 nnU-Net 資料集的 imagesTr 路徑
    nnunet_images_dir_train = "/home/ncku/Desktop/multi_nnunetv2/nnUNet/nnUNet_raw/Dataset520_trueLumen/imagesTr"
    # 執行拆分
    split_multichannel_to_nnunet_format(multichannel_dir_train, nnunet_images_dir_train)

## Step3

In [ ]:
import SimpleITK as sitk
import numpy as np
import os
from glob import glob
from tqdm import tqdm
from PIL import Image
from collections import defaultdict


def normalize_to_uint8(array: np.ndarray) -> np.ndarray:
    """
    將陣列標準化到 [0, 255] 範圍並轉換為 uint8 格式。
    
    Args:
        array (np.ndarray): 輸入陣列
    
    Returns:
        np.ndarray: uint8 格式的陣列
    """
    # 標準化到 [0, 1]
    array_min = array.min()
    array_max = array.max()
    
    if array_max - array_min > 0:
        normalized = (array - array_min) / (array_max - array_min)
    else:
        normalized = np.zeros_like(array)
    
    # 轉換到 [0, 255]
    return (normalized * 255).astype(np.uint8)


def save_channel_slices(image_sitk, output_dir: str, patient_id: str, channel_idx: int, channel_name: str):
    """
    儲存單通道影像的所有切片為 PNG 圖片。
    
    Args:
        image_sitk: SimpleITK 影像物件
        output_dir (str): 輸出資料夾路徑
        patient_id (str): 患者 ID（例如 patient0001）
        channel_idx (int): 通道索引（0, 1, 2）
        channel_name (str): 通道名稱（例如 'channel0'）
    """
    # 將影像轉換為 NumPy 陣列 (Z, Y, X)
    image_array = sitk.GetArrayFromImage(image_sitk)
    
    # 建立患者專屬的輸出資料夾
    patient_output_dir = os.path.join(output_dir, patient_id)
    os.makedirs(patient_output_dir, exist_ok=True)
    
    num_slices = image_array.shape[0]
    
    # 逐一儲存每個切片
    for slice_idx in range(num_slices):
        slice_array = image_array[slice_idx, :, :]
        
        # 標準化到 [0, 255]
        slice_uint8 = normalize_to_uint8(slice_array)
        
        # 轉換為 PIL Image 並儲存
        img = Image.fromarray(slice_uint8, mode='L')
        output_path = os.path.join(
            patient_output_dir, 
            f"slice_{slice_idx:04d}_ch{channel_idx}_{channel_name}.png"
        )
        img.save(output_path)


def convert_nnunet_format_to_png(input_dir: str, output_dir: str):
    """
    將 nnU-Net 格式的影像（多個單通道檔案）轉換為 PNG 圖片。
    每個患者每個切片會產生 3 張 PNG（對應 3 個通道）。
    
    Args:
        input_dir (str): 包含 nnU-Net 格式影像的資料夾路徑
        output_dir (str): PNG 圖片的輸出資料夾路徑
    """
    print("\n=== 開始轉換 nnU-Net 格式影像為 PNG ===")
    
    # 確保輸出資料夾存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 尋找所有影像檔案
    image_files = sorted(glob(os.path.join(input_dir, "patient*.nii.gz")))
    
    if not image_files:
        print(f"錯誤：在 '{input_dir}' 中找不到任何影像檔案。")
        return
    
    print(f"找到 {len(image_files)} 個影像檔案")
    
    # 將檔案按患者分組
    patient_files = defaultdict(list)
    for file_path in image_files:
        filename = os.path.basename(file_path)
        # 提取患者 ID（例如 patient0001）
        # 檔名格式：patient0001_0000.nii.gz, patient0001_0001.nii.gz, ...
        if "_" in filename:
            patient_id = filename.split("_")[0]
            patient_files[patient_id].append(file_path)
    
    print(f"識別到 {len(patient_files)} 個患者")
    
    # 通道名稱
    channel_names = ['channel0', 'channel1', 'channel2']
    
    success_count = 0
    fail_count = 0
    
    # 對每個患者處理
    for patient_id in tqdm(sorted(patient_files.keys()), desc="轉換患者"):
        try:
            files = sorted(patient_files[patient_id])
            
            # 檢查是否有 3 個通道
            if len(files) != 3:
                print(f"\n警告：{patient_id} 的檔案數量不是 3 個（實際: {len(files)}），跳過...")
                fail_count += 1
                continue
            
            # 處理每個通道
            for ch_idx, file_path in enumerate(files):
                # 讀取影像
                image_sitk = sitk.ReadImage(file_path, sitk.sitkFloat32)
                
                # 儲存切片
                save_channel_slices(
                    image_sitk, 
                    output_dir, 
                    patient_id, 
                    ch_idx, 
                    channel_names[ch_idx]
                )
            
            success_count += 1
            
        except Exception as e:
            print(f"\n處理患者 {patient_id} 時發生錯誤: {e}")
            fail_count += 1
            continue
    
    print(f"\n=== 轉換完成！===")
    print(f"成功: {success_count} 個患者")
    print(f"失敗: {fail_count} 個患者")
    print(f"PNG 圖片已儲存至: {output_dir}")
    
    # 顯示第一個患者的輸出範例
    if patient_files:
        first_patient = sorted(patient_files.keys())[0]
        first_patient_dir = os.path.join(output_dir, first_patient)
        if os.path.exists(first_patient_dir):
            png_files = sorted(glob(os.path.join(first_patient_dir, "*.png")))
            print(f"\n範例輸出（{first_patient}）:")
            print(f"  總共生成 {len(png_files)} 張 PNG")
            if png_files:
                print(f"  範例檔名:")
                for png_file in png_files[:6]:  # 顯示前 6 個
                    print(f"    - {os.path.basename(png_file)}")
                if len(png_files) > 6:
                    print(f"    ... (還有 {len(png_files) - 6} 張)")


def convert_original_and_nnunet_to_png(original_dir: str, nnunet_dir: str, 
                                       original_png_dir: str, nnunet_png_dir: str):
    """
    同時轉換原始單通道 CT 和 nnU-Net 格式的多通道 CT 為 PNG。
    
    Args:
        original_dir (str): 原始 CT 影像資料夾
        nnunet_dir (str): nnU-Net 格式影像資料夾
        original_png_dir (str): 原始 CT 的 PNG 輸出資料夾
        nnunet_png_dir (str): nnU-Net 格式的 PNG 輸出資料夾
    """
    # 轉換原始 CT 影像（如果需要）
    if os.path.exists(original_dir):
        print("\n【轉換原始 CT 影像】")
        convert_original_ct_to_png(original_dir, original_png_dir)
    
    # 轉換 nnU-Net 格式影像
    print("\n【轉換 nnU-Net 格式影像】")
    convert_nnunet_format_to_png(nnunet_dir, nnunet_png_dir)
    
    print("\n=== 所有轉換完成！===")


def convert_original_ct_to_png(input_dir: str, output_dir: str):
    """
    將原始單通道 CT 影像轉換為 PNG 圖片（可選功能）。
    
    Args:
        input_dir (str): 包含原始 CT 影像的資料夾路徑
        output_dir (str): PNG 圖片的輸出資料夾路徑
    """
    print("\n=== 開始轉換原始 CT 影像 ===")
    
    # 確保輸出資料夾存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 尋找所有 CT 影像檔案（只有 _0000 後綴的）
    image_files = sorted(glob(os.path.join(input_dir, "patient*_0000.nii.gz")))
    
    if not image_files:
        print(f"錯誤：在 '{input_dir}' 中找不到任何影像檔案。")
        return
    
    print(f"找到 {len(image_files)} 個原始 CT 影像檔案")
    
    for input_path in tqdm(image_files, desc="轉換原始 CT"):
        try:
            # 提取患者 ID（例如 patient0001）
            filename = os.path.basename(input_path)
            patient_id = filename.replace("_0000.nii.gz", "")
            
            # 讀取影像
            image_sitk = sitk.ReadImage(input_path, sitk.sitkFloat32)
            image_array = sitk.GetArrayFromImage(image_sitk)
            
            # 建立患者專屬的輸出資料夾
            patient_output_dir = os.path.join(output_dir, patient_id)
            os.makedirs(patient_output_dir, exist_ok=True)
            
            num_slices = image_array.shape[0]
            
            # 逐一儲存每個切片
            for slice_idx in range(num_slices):
                slice_array = image_array[slice_idx, :, :]
                
                # 標準化到 [0, 255]
                slice_uint8 = normalize_to_uint8(slice_array)
                
                # 轉換為 PIL Image 並儲存
                img = Image.fromarray(slice_uint8, mode='L')
                output_path = os.path.join(patient_output_dir, f"slice_{slice_idx:04d}.png")
                img.save(output_path)
            
        except Exception as e:
            print(f"\n處理檔案 {input_path} 時發生錯誤: {e}")
            continue
    
    print(f"原始 CT 影像轉換完成！PNG 圖片已儲存至: {output_dir}")


# --- 主程式執行區 ---
if __name__ == '__main__':
    # --- 設定您的資料夾路徑 ---
    # nnU-Net 格式的影像資料夾（包含 _0000, _0001, _0002）
    nnunet_images_dir_train = "/home/ncku/Desktop/multi_nnunetv2/nnUNet/nnUNet_raw/Dataset520_trueLumen/imagesTr"
    # PNG 輸出資料夾
    nnunet_png_dir = "/home/ncku/Desktop/multichannel_ct/png_multichannel_3"
    convert_nnunet_format_to_png(nnunet_images_dir_train, nnunet_png_dir)
